### Libraries Required

In [1]:
import pandas as pd
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import IntegerType, StringType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

### Initiate Spark Session

In [2]:
sc = SparkContext.getOrCreate()

if (sc is None):
    sc = SparkContext(master="local[2]", appName="Meal Recipe Collaborative Filtering")
spark = SparkSession(sparkContext=sc)

22/10/06 18:39:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/06 18:39:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
test_user_data = pd.read_csv('user_rating_test_data.csv', ',')
test_user_data['Recipe_Index'] = test_user_data.index
# test_user_data = test_user_data.dropna()
del test_user_data['Title']

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3398: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Input Format

input should be in the form of the table below User_ID, Recipe_Index, Individual's Recipe Rating

In [4]:
test_user_data = test_user_data.melt(id_vars = 'Recipe_Index', var_name='User_ID', value_name='Rating')
test_user_data = test_user_data[['User_ID', 'Recipe_Index', 'Rating']]
test_user_data = test_user_data.dropna()

test_user_data.head(5)

,User_ID,Recipe_Index,Rating
4,0,4,4.0
6,0,6,5.0
8,0,8,4.0
9,0,9,5.0
22,0,22,1.0


In [5]:
# turn pandas df into spark df for training
test_user_df = spark.createDataFrame(test_user_data)

# convert user_id to string
test_user_df = test_user_df \
    .withColumn('User_ID', test_user_df['User_ID'].cast(IntegerType()))

### Model Training

In [6]:
# train / test split
train, test = test_user_df.randomSplit([0.8, 0.2])

# define ALS model hyperparameters
als = ALS(maxIter=4, regParam=0.1, userCol="User_ID", itemCol="Recipe_Index", ratingCol="Rating",
          coldStartStrategy="drop")
model = als.fit(train)

22/10/06 18:40:02 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/10/06 18:40:02 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/10/06 18:40:02 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/10/06 18:40:02 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


#### Model Performance

In [7]:
# apply model to test
# predictions = model.transform(test)

# eval = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction")
# rmse = eval.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 3.9565706811683357


MSE is quite poor, suggesting that more data is required to provide more reliable predictions<br>
Ideally we should have below 1.0 RMSE

### Extracting Features (Pandas)

In [7]:
userRecs = model.recommendForAllUsers(50)

In [8]:
pd.set_option('display.max_colwidth', None)
user_predictions = userRecs.toPandas()

user_predictions['User_ID'] = user_predictions['User_ID'].astype(str)

In [9]:
user_predictions.head(10)

,User_ID,recommendations
0,12,"[(8, 5.376016139984131), (358, 5.023803234100342), (43, 5.023803234100342), (71, 5.023803234100342), (374, 5.023803234100342), (52, 5.023803234100342), (279, 4.97892951965332), (318, 4.954041957855225), (177, 4.8562774658203125), (219, 4.785390377044678), (384, 4.771224021911621), (229, 4.678145885467529), (134, 4.36651086807251), (133, 4.01904296875), (183, 4.01904296875), (368, 4.01904296875), (2, 4.01904296875), (264, 4.01904296875), (13, 4.01904296875), (315, 4.01904296875), (95, 4.01904296875), (29, 4.01904296875), (228, 3.9831433296203613), (435, 3.849364995956421), (319, 3.8318212032318115), (359, 3.8318212032318115), (514, 3.72397518157959), (537, 3.6805195808410645), (545, 3.6805195808410645), (351, 3.6805195808410645), (61, 3.529479503631592), (9, 3.5080208778381348), (189, 3.4907331466674805), (10, 3.4907331466674805), (207, 3.4907331466674805), (256, 3.4907331466674805), (76, 3.4907331466674805), (273, 3.4907331466674805), (320, 3.4907331466674805), (136, 3.4907331466674805), (80, 3.4907331466674805), (36, 3.4907331466674805), (21, 3.4807207584381104), (99, 3.1906204223632812), (424, 3.119018077850342), (597, 3.0619282722473145), (55, 2.9766366481781006), (532, 2.94441556930542), (379, 2.94441556930542), (244, 2.9198625087738037)]"
1,1,"[(234, 4.946934700012207), (54, 4.937650680541992), (123, 4.896336555480957), (170, 4.896336555480957), (348, 4.896336555480957), (248, 4.0796685218811035), (167, 4.07916259765625), (152, 3.9875729084014893), (450, 3.9501209259033203), (555, 3.707733154296875), (349, 3.645171642303467), (520, 3.5876309871673584), (378, 3.422382116317749), (6, 3.422382116317749), (517, 3.422382116317749), (258, 3.422382116317749), (423, 3.422382116317749), (484, 3.422382116317749), (480, 3.422382116317749), (561, 3.422382116317749), (116, 3.422382116317749), (448, 3.422382116317749), (276, 3.422382116317749), (206, 3.422382116317749), (291, 3.422382116317749), (403, 3.422382116317749), (559, 3.422382116317749), (215, 3.422382116317749), (298, 3.1325600147247314), (134, 3.0961716175079346), (153, 2.999960422515869), (601, 2.820251941680908), (350, 2.7947275638580322), (296, 2.7947275638580322), (124, 2.7947275638580322), (419, 2.7947275638580322), (539, 2.737905979156494), (562, 2.737905979156494), (375, 2.737905979156494), (109, 2.737905979156494), (201, 2.737905979156494), (401, 2.737905979156494), (364, 2.5513076782226562), (55, 2.5405144691467285), (456, 2.51198673248291), (9, 2.435243844985962), (137, 2.404531955718994), (369, 2.394360065460205), (446, 2.2357821464538574), (278, 2.2357821464538574)]"
2,13,"[(8, 4.96788215637207), (537, 4.940181255340576), (545, 4.940181255340576), (351, 4.940181255340576), (514, 4.719198703765869), (384, 4.406377792358398), (379, 3.9521446228027344), (532, 3.9521446228027344), (43, 3.8388102054595947), (52, 3.8388102054595947), (358, 3.8388102054595947), (71, 3.8388102054595947), (374, 3.8388102054595947), (279, 3.740809917449951), (229, 3.7381157875061035), (318, 3.5370681285858154), (219, 3.511699914932251), (177, 3.373915910720825), (315, 3.0710482597351074), (264, 3.0710482597351074), (183, 3.0710482597351074), (133, 3.0710482597351074), (13, 3.0710482597351074), (368, 3.0710482597351074), (2, 3.0710482597351074), (95, 3.0710482597351074), (29, 3.0710482597351074), (228, 2.992648124694824), (99, 2.869570732116699), (319, 2.7320306301116943), (359, 2.7320306301116943), (577, 2.59279465675354), (136, 2.565070152282715), (36, 2.565070152282715), (207, 2.565070152282715), (80, 2.565070152282715), (10, 2.565070152282715), (320, 2.565070152282715), (256, 2.565070152282715), (189, 2.565070152282715), (76, 2.565070152282715), (273, 2.565070152282715), (380, 2.5426597595214844), (20, 2.5426597595214844), (14, 2.366821527481079), (435, 2.345383405685425), (172, 2.052056312561035), (237, 2.052056312561035), (78, 2.052056312561035), (250, 2.052056312561035)]"
3,6,"[(280, 5.049895286560059), (74, 5.049895286560059), (156, 5.049895286560059),

### Extract Recommendations

Run this function to get the N-number of recommendations that should be shown to the user

extractRecommendations(ratings_df, predictions_df, user_id, num_of_recommendations)<br>

Where:<br><br>
<b>ratings_df</b><br> refers to the user data table from the database, including user_id, recipe_id, recipe_ratings<br>

<b>predictions_df</b><br> is the recomemndations for each user based on the trained model<br>

<b>user_id</b><br> being the unique user identifying number<br>

<b>num_of_recommendations</b><br> being the number of recommendations you wish to output<br>

In [13]:
def extractRecommendations(ratings_df, predictions_df, user_id, num_of_recommendations):
    
    predicted_recipes = []
    user_ratings = ratings_df[ratings_df['User_ID'] == user_id]['Recipe_Index'].tolist()
    
    for item in predictions_df[predictions_df['User_ID'] == user_id]['recommendations'].tolist()[0]:
        predicted_recipes.append(item[0])
        
    return [x for x in predicted_recipes if x not in user_ratings][:num_of_recommendations]
    
    
print(extractRecommendations(test_user_data, user_predictions, '11', 5))

[153, 134, 298, 72, 138]


In [25]:
def extractRecommendations(ratings_df, predictions_df, user_id, num_of_recommendations):
    
    predicted_recipes = []
    user_ratings = ratings_df[ratings_df['User_ID'] == user_id]['Recipe_Index'].tolist()
    
    for item in predictions_df[predictions_df['User_ID'] == user_id]['recommendations'].tolist()[0]:
        predicted_recipes.append(item[0])
        
    return [x for x in predicted_recipes if x not in user_ratings][:num_of_recommendations]
    
    
    
print(extractRecommendations(test_user_data, user_predictions, '11', 5))

[153, 134, 298, 72, 138]


In [33]:
user_list = []
for index, row in user_predictions.iterrows():
    user_list.append(row[0])

recommended_recipes = []
for user in user_list:
    recommended_recipes.append(extractRecommendations(test_user_data, user_predictions, user, 5))
    
output_df = pd.DataFrame({'User': user_list, 'Recommendations': recommended_recipes})

output_df.to_csv('user_recommendations.csv', sep=',', encoding='utf-8', index=False)